## Downloading all of the dataset

In [ ]:
# Download all of the dataset - Option 1
!curl -L -O https://isic-archive.com/api/v1/image/download?include=all

In [ ]:
# Download all of the dataset - Option 2
!nohup curl -L -O https://isic-archive.com/api/v1/image/download?include=all &

In [ ]:
# Download all of the dataset - Option 3
!nohup lftp -c 'set net:idle 10 
!      set net:max-retries 0
!      set net:reconnect-interval-base 3
!      set net:reconnect-interval-max 3
!      pget -n 10 -c "https://isic-archive.com/api/v1/image/download?include=all"' &

In [ ]:
# Download all of the dataset - Option 4

!lftp -c 'set net:idle 10
!      set net:max-retries 0
!      set net:reconnect-interval-base 3
!      set net:reconnect-interval-max 3
!      pget -n 10 -c "https://isic-archive.com/api/v1/image/download?include=all"'

In [ ]:
# Unzip Downloaded file

!unzip <downloaded_file_name> # Unzip the downloaded images

## Data Cleaning

In [2]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf

pd.options.display.max_columns = 100

from src.functions import preprocess_image, load_and_preprocess_image, create_X_y

In [18]:
## This code will only work if you have all the images already downloaded on your local drive
img_url = sorted(glob.glob('data/ISIC-images'+"/*.jpg"))
csv_names = [i for i in glob.glob('data'+'/*.csv')]
df = pd.read_csv('data/metadata.csv')

short=[]
for i in img_url:
    short.append(i[17:-4])
img_name = pd.Series(short,name='name')

df = pd.merge(df, img_name, on='name', how='inner')
df['image'] = img_url

df = df[['meta.clinical.diagnosis', 'image']]
df_na = df.copy()
df = df[df['meta.clinical.diagnosis'].notna()].reset_index(drop=True)
df.columns = ['diagnosis', 'image']
df['diagnosis'] = (df['diagnosis'] == 'melanoma').astype(int)

## Deleted null values from dataset

In [3]:
df_na[df_na['meta.clinical.diagnosis'].isna()]

,meta.clinical.diagnosis,image
257,NaN,data/ISIC-images/ISIC_0000257.jpg
454,NaN,data/ISIC-images/ISIC_0000454.jpg
9892,NaN,data/ISIC-images/ISIC_0009892.jpg
9893,NaN,data/ISIC-images/ISIC_0009893.jpg
9924,NaN,data/ISIC-images/ISIC_0009924.jpg
...,...,...
13786,NaN,data/ISIC-images/ISIC_0024202.jpg
13787,NaN,data/ISIC-images/ISIC_0024203.jpg
13788,NaN,data/ISIC-images/ISIC_0024204.jpg
13789,NaN,data/ISIC-images/ISIC_0024205.jpg


## Create Test Set and Save processed images

In [27]:
# Code to create X and y.
X, y = create_X_y(df)

In [22]:
# Save to local drive
np.save('data/100x100.npy', X)
np.save('data/diagnosis.npy', y)